In [40]:
import pandas as pd
import os
import json
import numpy as np


POSITION_BANK = ["President", "Chancellor", "Provost", "Director", "Dean", "Controller", "Trustee", "Member", "Regent", "Chairman", "Overseer", "Assistant", "Librarian", "Secretary", "Chaplain", "Minister", "Treasurer", "Senior Counsel", "General Counsel", "Legal Counsel", "University Counsel", "College Counsel", "Special Counsel", "Corporation Counsel", "Officer", "Chief", "Professor", "Commissioner", "Fellow", "Chairperson", "Manager", "Clergy", "Coordinator", "Auditor", "Governor", "Representative", "Stockbroker", "Advisor", "Commandant", "Rector", "Attorney", "Curator", "Clerk", "Department Head", "Pastor", "Head", "Comptroller", "Deputy", "Inspector General"]
#for classifying position as dean, administration
DEAN_WORDS = ["summer", "student", "faculty", "academic service", "academics", "academic program", "admissions", "admission", "enrollment", "student life", "housing", "academic support", "advising", "enrollment management", 
                       "student relations", "academic computing", "academic records", "student service", "student affairs", "student development", "registrar", "financial aid", "media service", "library service", "university librar",
                       "internation affair", "special program", "continuing education", "external relation", "development", "services"]

#for subinstituion
ADMINISTRATION_WORDS = ["academic service", "academics", "academic program", "admissions", "admission", "enrollment service", "student life", "housing", "academic support", "advising", "enrollment management", 
                       "student relations", "academic computing", "academic records", "student service", "student affairs", "student development", "registrar", "financial aid", "media service", "library service", "university librar"]


In [41]:
#define keywords
keywords = {
    "Arts and Sciences": ["arts and science", "letters and science", "arts, sciences, and letters", "arts, sciences", "arts & sciences", "letters, sciences", "arts and letters", "arts & letters"],
    "Engineering": ["engineering", "mines", "aerospace", "aeronautics"],
    "Applied Science": ["applied science"],
    
    "Computer Science": ["computer science", "computer,", "software engineering", "and computer", "computing"],
    "Data Science": ["data science", "artificial intelligence", "machine learning"],
    "Technology": ["technology"],
    "Information": ["information science", "information management", "information systems", "information stud", "library science", "and information", "library stud", "informatics", "school of information", 
                    "information and", "information school"],
    "Archictecure": ["architecture", "architectural", "urban design", "urban planning", "environmental design"],
    

    "Behavioral Science": ["psychology", "psychological", "human science", "human ecology", "behavioral", "human and", "childgood stud"],
    "Social Sciences": ["social science", "social and", "social ecology"],
    "Natural Sciences": ['natural science', "mathematics", "chemistry", "biology", "mathematical", "physical science", "life science", "biological science", 
                        "biochemistry", "natural and", "basic", "college of science", "school of science", ", science, and", ", science and", "physics"],
    "Environmental and Earth Science": ["environmental science", "forestry", "natural resource", "college of forest", 
                              "school of forest", "school of environment ", "environmental and", "geology", "oceanography", "earth science", "geophysics", "seismology", "volcanology", "earth and" 
                              "paleontology", "mineral science", "geoscience", "earth science", "earth studies", "atmospheric"],
    "Marine Science": ["marine science", "marine stud", "ocean science", "oceanography", "oceanographic", "ocean and"],

    
    "Business": ["business", "commerce", "mba", "college of economics", "and economics", "finance", "entrepreneurship", "school of management", "and management", "accounting", "agribusiness", "economic,", "accountancy"],
    "Hospitality": ["hospitality", "hotel administration", "hotel management", "restaurant management"],
    "Education": ["college of education", "school of education", "teaching", "teach", "curriculum development", "human development", "and education", ", education and"],
    "Criminal Justice": ["criminal justice", "criminology", "criminal", "forensics"],


    "Law": ["law", "legal studies"],
    "Policy and Affairs": ["public policy", "international policy", "public affairs", "international affairs", "environmental affairs", "urban affairs", "metropolitan affairs", "international service", 
                           "diplomacy", "diplomat", "international studies", "public administration", "urban policy", "global studies", "international relations", "social policy", "peace studies"],
    

    "Agriculture": ["agriculture", "agricultural", "agribusiness"],
    "Family and Consumer Science": ["family and", "home economics", "consumer science", "family science", "home"],
    "Culinary": ["culinary", "cooking", "gastronomy"],


    "Journalism": ["journalism"],
    "Communication": ["communication"],
    "Liberal Arts": ["liberal art", "liberal studies", "interdisciplinary studies", "general studies", "liberal and"],
    "Humanities": ["humanities", "history", "historical", "philosophy", "literature", "political"],
    "Fine Arts": ["fine arts", "music", "theater", "creative", "theatre", "film", "performing art", "the arts", "design art", "dramatic arts", "dance",
                   "visual arts", "school of drama", "recording arts", "ceramics", "fine and applied art", "school of design", "art and design", "fine", 
                   "cinema", "art ", "art, ", "studio art", "media art", "arts and performance", "drama", "arts and media"],
    "Language and Culture": ["languages", "lingusitics", "cultures", "translational studies", "intercultural studies", "ethnic studies", "hebraic studies", "cultural stud", "asian studies", "culture "],


    "Nursing": ["nursing"],
    "Physical Therapy and Nutrition": ["physical therapy", "nutrition", "physiotherapy", "nutritional therapy", "rehab", "human performance"],
    "Medical": ["medical", "pharmacy", "surgeons", "school of medicine", "college of medicine", "podiatric medicine", "osteopathic medicine"],
    "Pharmacy": ["pharmacy", "pharmacology"],
    "Dentistry" : ["dentistry", "dental"],
    "Optometry" : ["optometry", "optics", "optical science"],
    "Veterinary" : ['veterinary'],
    "Audiology": ["audiology"],
    "Gerontology": ["gerontology"],
    "Pathology": ["pathology"],
    "Health Science": ["health science", "public health", "health profession", "college of health", "school of health", "biomedical science", "and health", "allied health", "physical education",
                        "health and", "health studies", "genome"],


    "Religious Studies": ["religious", "theology", "divinity", "biblical", "theological", "religion", "seminary", "religous", "christ college"],
    "Social Work": ["social work", "social welfare", "community service", "human service", "social service", "public service"],
    

    "Graduate": ["graduate school", "graduate studies", "graduate college", "college of graduate", "school of graduate",  "advancing", "advanced study", "graduate division", "graduate and professional studies", "doctoral", "graduate study", "graduate "],
    "Continued Studies": ["extended studies", "extended education", "continuing education", "continued education", "continuing studies", "lifelong learning", "professional programs", "extended learning",
                           "adult education", "online learning", "adult student", "lifelong education", "continuing and", "university extension", "distance learning", "continuous education"],
    "Professional Studies": ["professional studies", "professional development", "professional education", "school of professions"],
    "Administration": ["academic service", "academics", "academic program", "admissions", "admission", "enrollment service", "student life", "housing", "academic support", "advising", "enrollment management", 
                       "student relations", "academic computing", "academic records", "student service", "student affairs", "student development", "registrar", "financial aid", "media service", "library service", 
                       "university librar", "information service", "information access", "library access", "residence", "academic affairs", "library affairs", "academic succes", "first year student",
                       "undergraduate studies", "undergraduate education"],
    "Honor College": ["honor college", "honors college", "scholar program"],

    "Satellite Campus": ["campus"],
    "Foundation": ["foundation"],
    "Boards": ["board of", "regents", "trustees", "council of"],
    "Throwaway": ["placeholder value"],
    "Hospital": ["hospital"],
    "University College": ["university college"]
}

In [42]:
path_connected_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\"
path_temp_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\temporaryData\\"
year = "1999"

path_read = f"{path_connected_data}{year}_split_positions.csv"
cleaned_path = f"{path_connected_data}{year}_cleaned_positions.csv"
president_path = f"{path_temp_data}{year}_presidents.csv"
provost_path = f"{path_temp_data}{year}_provost.csv"
dean_path = f"{path_temp_data}{year}_dean.csv"
vice_presidents_path = f"{path_temp_data}{year}_vice_presidents.csv"
grouped_subinstitution_path = f"{path_connected_data}{year}_grouped.csv"

In [43]:
#Put every column from the xlsx into a df
def parse_columns(path, size):
    df = pd.read_csv(path)
    all_columns = df.iloc[:, :size]
    return all_columns


#Retrieve specific column by index
def retrieve_specific_column(df, index):
    return df.iloc[:, index]

#Extract the names of all the institutions for validation
def extract_institutions(df):
    institution_list = []
    for index, row in df.iterrows():
        if row["Institution"] not in institution_list:
            institution_list.append(row.iloc[2])
    return institution_list


def extract_first_member(df):
    first_member_df = []
    previous_institution = None
    for index, row in df.iterrows():
        current_institution = row["Institution"]
        if previous_institution is None or current_institution != previous_institution:
            first_member_df.append(row)
        previous_institution = current_institution
    return pd.DataFrame(first_member_df)

#extract first person from each institution
def extract_first_member_exclude_string(df, string):
    string = string.lower()
    first_member_df = []
    previous_institution = None
    for index, row in df.iterrows():
        current_institution = row["Institution"]
        # Check if the value at index 2 has changed, including handling NaN for the first row
        if previous_institution is None or current_institution != previous_institution:
            if string not in row.iloc[1].lower():
                first_member_df.append(row)
        previous_institution = current_institution
    return pd.DataFrame(first_member_df)

#Replace all the entries of a column with a standardized value (for president)
def replace_values(df, string):
    df.iloc[:,1] = string
    return df 

#Find the institutions that were left out
def find_missing_institutions(institutions, df):
    lowercase_df = df.iloc[:,2].str.lower()
    missing_institutions = []
    for institution in institutions:
        if institution.lower() not in lowercase_df.values:
            missing_institutions.append(institution)
    return missing_institutions


def rename_previous_value(df):
    df.rename(columns={'Previous_Value': 'Fixed Position'}, inplace = True)
    df["Fixed Position"] = ""
    return pd.DataFrame(df)


#validate university list
def count_universities(full_df):
    institutions = full_df["Institution"].unique()
    return list(institutions)

In [44]:
full_dataframe = parse_columns(path_read, 8)
grouped_dataframe = parse_columns(grouped_subinstitution_path, 3)

In [45]:
#Presidents
def extract_presidents(df, value):
    member_list = []
    for index, row in df.iterrows():
        if row.iloc[value] == "President":
            member_list.append(row['Name'])
        elif row.iloc[value] == "Chancellor":
            member_list.append(row['Name'])
    return member_list

#check if the first person is president and the second is the chancellor (need to decide how to handle these two)
def extract_president_and_chancellor(president_df, df):
    previous_institution = None
    for index, row in df.iterrows():
        current_institution = row.iloc[2]
        if previous_institution is None or current_institution != previous_institution:
            if index + 1 < len(df):
                next_row = df.iloc[index + 1]
                if "president" in row.iloc[1].lower() and "chancellor" == next_row.iloc[1].lower():
                    president_df = pd.concat([president_df, next_row.to_frame().T], ignore_index=True)
        previous_institution = current_institution
    return president_df

#remove any non presidents/chancellors
def clean_president_list(df):
    cleaned_df = []
    for index, row in df.iterrows():
        if 'president' in row.iloc[1].lower() or 'chancellor' in row.iloc[1].lower() or 'director' in row.iloc[1].lower() or 'superintendent' in row.iloc[1].lower():
            #Don't want vice president of the university, but exceptions
            if 'vice president' not in row.iloc[1].lower():
                cleaned_df.append(row)
            elif row.iloc[1].lower().count('president') >= 2:
                cleaned_df.append(row)
            elif 'vice president' in row.iloc[1].lower() and 'chancellor' in row.iloc[1].lower():
                cleaned_df.append(row)
    cleaned_df = pd.DataFrame(cleaned_df)
    cleaned_df = cleaned_df.map(lambda x: x.title() if isinstance(x, str) else x)

    # cleaned_df = cleaned_df.drop_duplicates(subset = ["Name"], keep = "first")
    cleaned_df = cleaned_df.drop_duplicates(subset = ["Institution"], keep = "first")
    return cleaned_df


def create_presidents_df(full_df):
    # all_institutions = extract_institutions(full_df)
    presidents_initial = extract_first_member(full_df)
    presidents_cleaned = clean_president_list(presidents_initial)
    presidents_cleaned = extract_president_and_chancellor(presidents_cleaned, full_df)
    return pd.DataFrame(presidents_cleaned)

def determine_missing(university_list, presidents_df):
    president_institutions = list(presidents_df["Institution"])
    university_list_normalized = [uni.strip().lower() for uni in university_list]
    president_institutions_normalized = [president.strip().lower() for president in president_institutions]
    missing_list = [uni for uni in university_list_normalized if uni not in president_institutions_normalized]
    return missing_list

In [46]:
#Provost
#Extract every row with the substring provost in the title
def extract_provost(df):
    provost_df = []
    for index, row in df.iterrows():
        if 'provost' in row.iloc[1].lower():
            # print(row)
            provost_df.append(row)
    provost_df = pd.DataFrame(provost_df)
    return provost_df


def create_provost_df(full_df):
    provost_initial = extract_provost(full_df)
    provost_cleaned = extract_first_member_exclude_string(provost_initial, "associate provost")
    return pd.DataFrame(provost_cleaned)

In [47]:
#Deans
#more to add here, but probably better to finish subinstitutions problem before doing this.
def extract_deans(df):
    dean_df = []
    for index, row in df.iterrows():
        if ('dean' in row.iloc[1].lower()) and "assistant to the dean" not in row.iloc[1].lower():
            dean_df.append(row)
    dean_df = pd.DataFrame(dean_df)
    return dean_df

def assign_administrative_deans(df):
    for index, row in df.iterrows():
        position = row["Position"]
        subinstitution = row["SubInstitution"]
        # if type(subinstitution) != float or "of the college" in position.lower():
        if "of the college" in position.lower():
            continue
        elif type(subinstitution) == float:
            df.at[index, "Fixed Position"] = "Dean, " + "Administration"
        else:
            count = 0
            #check keywords in the position column
            for keyword in DEAN_WORDS:
                if keyword in position.lower():
                     count+=1
            if count > 0:
                df.at[index, "Fixed Position"] =  "Dean, " + "Administration"
                continue
            count = 0
            #check keywords in the subinstitution column
            for keyword in ADMINISTRATION_WORDS:
                if keyword in subinstitution.lower():
                    count+=1
            if count > 0:
                df.at[index, "Fixed Position"] =  "Dean, " + "Administration"
                continue
    return pd.DataFrame(df)

def assign_school_deans(grouped_df, full_df):
    grouped_dict = grouped_df.set_index(['Institution', 'SubInstitution'])['Category'].to_dict()
    for index, row in full_df.iterrows():
        position = row["Position"]
        subinstitution = row["SubInstitution"]
        academic_no_institution = "academic dean" in position.lower() and type(subinstitution) == float
        academic_with_institution = "academic dean" in position.lower() and type(subinstitution) == str
        key = (row["Institution"], row["SubInstitution"])

        if "dean of the college" in position.lower():
            full_df.at[index, "Fixed Position"] = "Dean of the College"
            continue
        elif academic_no_institution:
            full_df.at[index, "Fixed Position"] = "Dean of the College"
            continue
        elif academic_with_institution:
            full_df.at[index, "Fixed Position"] = "Dean, " +  grouped_dict[key]
            continue

        if key in grouped_dict:
            full_df.at[index, "Fixed Position"] = "Dean, " +  grouped_dict[key]
    return pd.DataFrame(full_df)    
    

def create_deans_df(grouped_df, full_df):
    full_df = rename_previous_value(full_df)
    deans_initial = extract_deans(full_df)
    deans_clean = assign_administrative_deans(deans_initial)
    deans_assigned = assign_school_deans(grouped_df, deans_clean)
    return pd.DataFrame(deans_assigned)

In [48]:
#Vice Presidents
def extract_vice_presidents(df):
    vp_df = []
    for index, row in df.iterrows():
        if 'vice president' in row.iloc[1].lower():
            vp_df.append(row)
    return pd.DataFrame(vp_df)

def create_vice_presidents_df(full_df):
    vice_presidents = extract_vice_presidents(full_df)
    return vice_presidents

In [49]:
df = parse_columns(path_read, 8)
presidents_df = create_presidents_df(df)
university_list = count_universities(df)
missing_presidents = determine_missing(university_list, presidents_df)

for inst in missing_presidents:
    print(inst.title())
presidents_df.to_csv(president_path, index = False)


Tennessee Board Of Regents
Montana University System
State University Of New York System
Texas A&M University Baylor College Of Dentistry
Texas A&M University Galveston
The University Of North Carolina System


In [50]:
provost_df = create_provost_df(df)
university_list = count_universities(df)
missing_provost = determine_missing(university_list, provost_df)

for inst in missing_provost:
    print(inst.title())

provost_df.to_csv(provost_path, index = False)

Agnes Scott College
Albion College
Allegheny College
Amherst College
Andrews University
Aquinas College
Austin Peay State University
Baker University
Baldwin Wallace College
Bard College
Bates College
Beaver College
Bellarmine College
Beloit College
Boston College
Bowdoin College
Brenau University
Brigham Young University
Bucknell University
Butler University
California State University System
California Maritime Academy
California Polytechnic State University San Luis Obispo
California State Polytechnic University Pomona
California State University, Channel Islands
California State University Dominguez Hills
California State University Fullerton
California State University San Bernardino
California State University San Marcos
Sonoma State University
Carleton College
Carthage College
Centre College
Chestnut Hill College
Christian Brothers University
The Citadel
City University Of New York
City University Of New York
Claremont Mckenna College
Clarkson University
Colby College
College Of

In [51]:
deans_df = create_deans_df(grouped_dataframe, full_dataframe)
deans_df.to_csv(dean_path, index = False)